<a href="https://www.kaggle.com/code/josemariamartin/acuerdos-marco-2020?scriptVersionId=100587208" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Análisis de Acuerdos Marco 2020

# load data 
Load data from https://www.hacienda.gob.es/es-ES/GobiernoAbierto/Datos%20Abiertos/Paginas/CCEContratosbasadosacuerdosmarco.aspx

In [ ]:
!pip install bs4

In [ ]:
import requests
import os
from bs4 import BeautifulSoup

def get_html(url):
    response = requests.get(url)
    return response.text

url_contratos = "https://www.hacienda.gob.es/es-ES/GobiernoAbierto/Datos%20Abiertos/Paginas/CCEContratosbasadosacuerdosmarco.aspx"
def load_cont(url):
    html = get_html(url)
    soup = BeautifulSoup(html, 'lxml')

    title = soup.title
    print(title)
    links = []
    for a in soup.findAll('a',href=True):
        if 'atom' in a['href']:
            links.append(''.join(('https://www.hacienda.gob.es'+a['href'])))
            
    return links
            
enlaces = load_cont(url_contratos)
os.chdir(r'/kaggle/working')
for enlace in enlaces:
        print(enlace)
        x = enlace.rfind("/")+1
        file_name = enlace[x:]
        print(file_name)
        r = requests.get(enlace)  
        with open(file_name, 'wb') as f:
            f.write(r.content)
#        file = get_html(enlace)
#        file.save


In [ ]:
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(filename)

## Import libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xml.etree.ElementTree as et
import sys

import matplotlib.pyplot as plt
import seaborn as sns


## Load data

Data improted from https://www.hacienda.gob.es/es-ES/GobiernoAbierto/Datos%20Abiertos/Paginas/CCEContratosbasadosacuerdosmarco.aspx

TODO: Automatically read all files in this page (https://www.kaggle.com/artemtprv/load-external-data)

In [ ]:

# Define data fields
df_cols = ["id", "Title","OrgName","DIR3", "Fecha", "AM", "Adj.", "CIF","Importe"]

rows = []
ns = {'atom': 'http://www.w3.org/2005/Atom',
      'cac':"urn:dgpe:names:draft:codice:schema:xsd:CommonAggregateComponents-2",
      'cbc':"urn:dgpe:names:draft:codice:schema:xsd:CommonBasicComponents-2",
      'cbc-place-ext':"urn:dgpe:names:draft:codice-place-ext:schema:xsd:CommonBasicComponents-2",
      'cac-place-ext':"urn:dgpe:names:draft:codice-place-ext:schema:xsd:CommonAggregateComponents-2"}

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
def parse_XML(xml_file): 
        xtree = et.parse(xml_file)
        xroot = xtree.getroot()
        rows = []
        
        for entry in xroot.findall("atom:entry",ns):            
            s_id = entry.find("atom:id",ns).text
            s_title = entry.find("atom:title",ns).text

            for cfs in entry.find("cac-place-ext:ContractFolderStatus",ns):
               # for child in cfs:
               #         print(child.tag, child.attrib, child.text) 
                for names in cfs.findall("./cbc:Name",ns):              
                    s_am = names.text                         #CAREFUL, for whatever reason it doesn't find cac:ProcurementProject
                for dates in cfs.findall("./cbc:AwardDate",ns):              
                    s_fecha = dates.text                      #CAREFUL, for whatever reason it doesn't find cac:TenderResult
                
                for party in cfs.findall("cac:Party",ns):
                    s_organismo = (party.find("cac:PartyName/cbc:Name",ns).text)
                    if not party.find("cac:PartyIdentification/cbc:ID",ns) is None:
                        s_dir3 = (party.find("cac:PartyIdentification/cbc:ID",ns).text)
                    else:
                        s_dir3=""
                for winningparty in cfs.findall("cac:WinningParty",ns):
                    s_adj = (winningparty.find("cac:PartyName/cbc:Name",ns).text)
                    if not winningparty.find("cac:PartyIdentification/cbc:ID",ns) is None:
                        s_cif = (winningparty.find("cac:PartyIdentification/cbc:ID",ns).text)
                    else:
                        s_cif=""
                for project in cfs.findall("cac:AwardedTenderedProject",ns):
                    s_imp = (project.find("cac:LegalMonetaryTotal/cbc:TaxExclusiveAmount",ns).text)
            rows.append({"id": s_id, "Title":s_title ,"OrgName":s_organismo, "DIR3":s_dir3, "Fecha":s_fecha, "AM":s_am, "Adj.":s_adj,"CIF":s_cif,  "Importe":s_imp})
        
        out_df = pd.DataFrame(rows, columns=df_cols)
       # print(out_df)
        return out_df    
        #return rows

# Read all the files         
table =[]
df = pd.DataFrame([], columns=df_cols)
import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        if 'atom' in filename:
            try: 
                dft = parse_XML(os.path.join(dirname, filename))
                frames = [df, dft]
                df = pd.concat(frames)
            except:
                print("Fallo al cargar "+filename)
#print("Data lenght: " + str(len(df)))
#print(df.iloc[0])


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Exploratory Data Analysis

In [ ]:
df.head()

In [ ]:
print(df.shape)
print(df.columns)

In [ ]:
print(df.info())

### Clean and add data fields

In [ ]:
df['Importe'] = df['Importe'].astype('float64')

In [ ]:
df['Fechadate'] = pd.to_datetime(df['Fecha'].str[:10], format='%Y-%m-%d', errors='coerce')

In [ ]:
df['Imp(K€)'] = df['Importe'].apply(lambda x: "{:.1f}".format((x/1000))).astype('float64')

In [ ]:
df['Mes'] = df['Fecha'].str[5:7].astype('int64')
df['Año'] = df['Fecha'].str[0:4].astype('int64')

In [ ]:
df['AM_long'] = df['AM']
df['AM'] = df['AM_long'].str[0:7]

In [ ]:
df.head()

In [ ]:
print(df.info())

In [ ]:
df.describe()

## AM analysis

In [ ]:
df['AM'].value_counts().sort_index()


In [ ]:
df['Año'].value_counts().sort_index()

In [ ]:
df['Mes'].value_counts().sort_index()

In [ ]:
df['OrgName'].value_counts().sort_index()

In [ ]:
df['DIR3'].value_counts().sort_index()

In [ ]:

ImporteporAM = df.groupby('AM')['Imp(K€)'].sum()
ImporteporAM.head()

In [ ]:

columns_to_show = ['Importe','Imp(K€)']

df.groupby(['AM'])[columns_to_show].agg([np.mean, np.std, np.min, 
                                            np.max, np.sum])

In [ ]:
# Setting the size of the figure
plt.figure(figsize=(15, 10))

#Plotting the  line 
sns.lineplot(x=df['Mes'], y=df['Imp(K€)'], color='b', label='total')

plt.title('Evolución importe contratos')

plt.xlim(1, 12)

plt.legend();

In [ ]:
plt.figure(figsize=(15,10))
sns.barplot(x=ImporteporAM.index, y=ImporteporAM)
plt.xticks(rotation= 90)
plt.xlabel('AM')
plt.ylabel('Imp(K€)')
plt.title('Importe por AM')

**Acuerdos Marco de un organismo (BNE-E00123904)**

In [ ]:
pd.set_option('display.max_colwidth', None)
columns_to_show = [ "AM","Title", "Fechadate",  "Adj.",'Imp(K€)' ]
df[df['DIR3']=='E00123904'][columns_to_show].sort_values('Imp(K€)', ascending=False)

In [ ]:
pd.set_option('display.max_colwidth', None)
columns_to_show = [ "AM","Title", "Fechadate",  "Adj.",'Imp(K€)' ]
df[df['OrgName'].str.contains("BIBLIOTECA")][columns_to_show].sort_values(['AM','Imp(K€)'], ascending=False)

**AM Sanidad - E05070101**

In [ ]:
pd.set_option('display.max_colwidth', None)
columns_to_show = [ "AM","Title", "Fechadate",  "Adj.",'Imp(K€)' ]
df[df['DIR3']=='E05070101'][columns_to_show].sort_values(['AM','Imp(K€)'], ascending=False)

In [ ]:
pd.set_option('display.max_colwidth', None)
columns_to_show = [ "AM","Title", "Fechadate",  "Adj.",'Imp(K€)' ]
df[df['OrgName'].str.contains("SANIDAD")][columns_to_show].sort_values(['AM','Imp(K€)'], ascending=False)


# AM 26 - Servicios de desarrollo de sistemas...

**Importe anual por adjudicatario**

In [ ]:
pd.options.display.max_rows = None
contratos_AM26 = df[df['AM']=='26/2015']
columns_to_show = ['Importe','Imp(K€)']

contratos_AM26.groupby(['Adj.','Año'])[columns_to_show].sum().sort_values('Imp(K€)', ascending=False)

**Importe anual por Organismo**

In [ ]:
columns_to_show = ['Importe','Imp(K€)']

contratos_AM26.groupby(['OrgName','DIR3'])[columns_to_show].sum().sort_values('Imp(K€)', ascending=False)

**AM26/2015 Ministerio Sanidad**

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = None
columns_to_show = [ "id","Title", "Fechadate",  "Adj.",'Imp(K€)' ]
contratos_AM26[contratos_AM26['OrgName']=='MINISTERIO DE SANIDAD'][columns_to_show].sort_values('Imp(K€)', ascending=False)

** AM26/2015 SUBDIRECCIÓN GENERAL DE LA OFICINA PRESUPUESTARIA Y GESTIÓN ECONÓMICA . MPTFP**

In [ ]:
pd.set_option('display.max_colwidth', None)
columns_to_show = [ "Title","Fechadate",  "Adj.",'Imp(K€)' ]
contratos_AM26[contratos_AM26['DIR3']=='EA0020006'][columns_to_show].sort_values('Imp(K€)', ascending=False)

## SUBSECRETARÍA DE HACIENDA

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = None
columns_to_show = [ "id","Title", "Fechadate",  "Adj.",'Imp(K€)' ]
contratos_AM26[contratos_AM26['OrgName'].str.contains('SUBSECRETARÍA DE HACIENDA')][columns_to_show].sort_values('Imp(K€)', ascending=False)

# AM13 - Sistemas

In [ ]:
pd.options.display.max_rows = None
contratos_AM13 = df[df['AM'].str.contains("13/")]
columns_to_show = ['Importe','Imp(K€)']

contratos_AM13.groupby(['Adj.','Año'])[columns_to_show].sum()

In [ ]:
np.set_printoptions(threshold=np.inf)
columns_to_show = ['Importe','Imp(K€)']

contratos_AM13.groupby(['OrgName'])[columns_to_show].sum().sort_values('Imp(K€)', ascending=False)

In [ ]:
contratos_AM13.groupby(['Año'])[columns_to_show].sum()

In [ ]:
contratos_AM13['Año'].value_counts().sort_index()

In [ ]:
pd.set_option('display.max_colwidth', None)
columns_to_show = [ "Title","Fechadate",  "Adj.","OrgName","DIR3",'Importe' ]
contratos_AM13[contratos_AM13['Año']==2020][columns_to_show].sort_values('Importe', ascending=False)

In [ ]:
pd.set_option('display.max_colwidth', None)
columns_to_show = [ "Title","Fechadate",  "Adj.","DIR3",'Imp(K€)' ]
contratos_AM13[contratos_AM13['OrgName'].str.contains('BIBLIOTECA')][columns_to_show].sort_values('Imp(K€)', ascending=False)


In [ ]:
contratos_AM10 = df[df['AM'].str.contains("10/")]
columns_to_show = ['Importe','Imp(K€)']

contratos_AM10.groupby(['Adj.'])[columns_to_show].sum().sort_values('Imp(K€)', ascending=False)

# AM 23/2017 de electricidad

In [ ]:
contratos_AM23 = df[df['AM'].str.contains("23/")]
columns_to_show = ['Importe','Imp(K€)']

contratos_AM23.groupby(['Adj.'])[columns_to_show].sum().sort_values('Imp(K€)', ascending=False)

In [ ]:
contratos_AM23.groupby(['Año'])[columns_to_show].sum()

In [ ]:
contratos_AM23['Año'].value_counts().sort_index()